In [1]:
import pandas as pd
import numpy as np
import pickle as pk

In [62]:
file_name = 'holo_dirty_sample'
df = pd.read_csv(file_name + '.csv')
df['behavior'] = np.zeros(len(df)).astype(np.int)

In [63]:
used_id = set()
used_id = set(list(df['Vehicle_ID']))
max_l = dict()
max_w = dict()
for i in range(len(df)):
    r = df.iloc[i]
    v_id = r.Vehicle_ID
    
    if v_id not in max_w.keys():        
        max_w[v_id] = r.v_Width
        max_l[v_id] = r.v_length
    else:
        max_w[v_id] = max(max_w[v_id], r.v_Width)
        max_l[v_id] = max(max_l[v_id], r.v_length)
        
# for v_id in max_l.keys():
#     if max_l[v_id] < 4:
#         max_l[v_id] = 4.5
#     if max_w[v_id] < 1.5:
#         max_w[v_id] = 2
    

In [64]:
max_l

{0: 4.8700000000000001,
 611: 6.1980000000000004,
 919: 0.35299999999999998,
 928: 0.20000000000000001,
 2532: 4.9530000000000003,
 3131: 4.4409999999999998,
 3294: 4.3479999999999999,
 3478: 1.4199999999999999,
 4283: 0.20000000000000001,
 4627: 0.20000000000000001,
 4657: 0.20000000000000001,
 4751: 0.20000000000000001,
 4895: 0.20000000000000001,
 4976: 0.20000000000000001,
 5000: 0.20000000000000001,
 5172: 0.20000000000000001,
 5173: 0.20000000000000001,
 5376: 0.20000000000000001,
 5378: 0.20000000000000001,
 5566: 0.20000000000000001,
 5567: 0.20000000000000001,
 5785: 0.20000000000000001,
 5818: 0.20000000000000001,
 5957: 0.20000000000000001,
 6001: 0.38500000000000001,
 6459: 0.20000000000000001,
 7524: 0.503,
 9182: 0.20000000000000001,
 9319: 4.242,
 9522: 0.20000000000000001}

In [60]:
dT = 0.1
x = list()
y = list()
vehicles = dict()
show_up = set()
vel_sum = 0

id_map = dict()
discard_id = set()
id_cnt = 1
r_id_map = dict()

for i in range(len(df)):
    r = df.iloc[i]
    v_id = r.Vehicle_ID
    
    df.at[i,'v_Width'] = max_w[v_id]
    df.at[i,'v_length'] = max_l[v_id]
    
    if v_id in id_map:
        v_id = id_map[v_id]
    elif v_id in discard_id:
        while id_cnt in used_id or id_cnt in discard_id:
            id_cnt += 1
        id_map[v_id] = id_cnt
        r_id_map[id_cnt] = v_id
        v_id = id_map[v_id]
        
    df.at[i,'Global_Time'] = df.at[i,'Frame_ID'] * 100
    df.at[i,'Local_X'] = df.at[i,'Global_X'] - df.at[0, 'Global_X']
    df.at[i,'Local_Y'] = df.at[i,'Global_Y'] - df.at[0, 'Global_Y']
    
    if v_id not in vehicles.keys():
        vehicles[v_id] = i
        df.at[i,'v_Acc'] = 0
    else:
        df.at[i,'v_Acc'] = df.at[i,'v_Vel'] - df.at[vehicles[v_id], 'v_Vel']

    show_up.add(v_id)

    if i == len(df)-1 or r.Global_Time != df.iloc[i+1].Global_Time:
        for v_id in vehicles.keys():
            if v_id not in show_up:
                vehicles.remove(v_id)
                discard_id.add(v_id)
                id_map.remove(r_id_map(v_id))
                print(v_id)

In [55]:
len(discard_id)

0

In [45]:
df[['Local_X', 'Local_Y', 'Global_X', 'Global_Y', 'v_Vel', 'v_Acc','v_length', 'v_Width', 'Space_Headway' ]] *= 3.28

In [46]:
columnsTitles=["Local_X","Local_Y"]
df[["Local_X","Local_Y"]]=df.reindex(columns=columnsTitles)
df['Local_X'] = -df['Local_X']

In [47]:
df[:10]

,Unnamed: 0,Global_Time,Frame_ID,Total_Frame,Vehicle_ID,Local_X,Local_Y,Global_X,Global_Y,v_length,...,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location,behavior
0,0,100,1,39005,0,-0.00000,0.00000,1.530245e+06,1.453661e+07,15.97360,...,NaN,5.0,1,1.0,7524,0,-1.06600,-0.020,sunhe_t3,0
1,1,100,1,780,2532,185.10680,109.52904,1.530060e+06,1.453672e+07,16.24584,...,NaN,5.0,1,2.0,9182,611,43.56496,0.838,sunhe_t3,0
2,2,100,1,49,928,314.05016,131.12456,1.529931e+06,1.453674e+07,14.76000,...,NaN,5.0,1,1.0,0,0,0.00000,0.000,sunhe_t3,0
3,3,100,1,86,919,-382.86784,-142.75872,1.530628e+06,1.453646e+07,14.76000,...,NaN,5.0,1,2.0,9319,0,-2.31568,-0.120,sunhe_t3,0
4,4,100,1,1812,611,-60.46680,4.98560,1.530306e+06,1.453661e+07,20.32944,...,NaN,5.0,1,2.0,2532,9319,-7.14056,-0.320,sunhe_t3,0
5,5,100,1,3020,9319,-194.47120,-65.95752,1.530440e+06,1.453654e+07,13.91376,...,NaN,5.0,1,2.0,611,919,-16.82640,-3.222,sunhe_t3,0
6,6,100,1,3067,9182,238.85288,83.89584,1.530006e+06,1.453669e+07,14.76000,...,NaN,5.0,1,2.0,9522,2532,10.15160,1.038,sunhe_t3,0
7,7,100,1,699,9522,293.72072,94.62472,1.529951e+06,1.453670e+07,14.76000,...,NaN,5.0,1,2.0,0,9182,0.00000,0.000,sunhe_t3,0
8,8,100,1,601,7524,258.08024,102.94280,1.529987e+06,1.453671e+07,14.76000,...,NaN,5.0,1,2.0,0,0,0.00000,0.000,sunhe_t3,0
9,9,200,2,39005,0,0.42968,-1.07584,1.530245e+06,1.453661e+07,15.97360,...,NaN,5.0,1,1.0,7524,0,-1.06600,-0.020,sunhe_t3,0


In [48]:
df.to_csv('holo_sample_cleaned.csv')